<a href="https://colab.research.google.com/github/Larai15/kubernetes/blob/master/Copy_of_In_class_exercise_05.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **The fifth in-class-exercise**

## **1. Rule-based information extraction**

Use any keywords related to data science, natural language processing, machine learning to search from google scholar, get the **titles** of 100 articles (either by web scraping or manually) about this topic, define a set of patterns to extract the research questions/problems, methods/algorithms/models, datasets, applications, or any other important information about this topic. 

In [0]:
import urllib
from bs4 import BeautifulSoup
import requests
text = 'nlp'
text = urllib.parse.quote_plus(text)

url = 'https://scholar.google.com/scholar?q=' + text

response = requests.get(url)


soup = BeautifulSoup(response.text, 'lxml')
# print(response.text)
for g in list(soup.find_all(class_='gs_rt'))[:100]:
    print(g.text)
    print('-----')

## **2. Domain-specific information extraction**

For the legal case used in the data cleaning exercise: [01-05-1 Adams v Tanner.txt](https://raw.githubusercontent.com/unt-iialab/INFO5731_Spring2020/master/In_class_exercise/01-05-1%20%20Adams%20v%20Tanner.txt), use [legalNLP](https://lexpredict-lexnlp.readthedocs.io/en/latest/modules/extract/extract.html#nlp-based-extraction-methods) to extract the following inforation from the text (if the information is not exist, just print None):

(1) acts, e.g., “section 1 of the Advancing Hope Act, 1986”

(2) amounts, e.g., “ten pounds” or “5.8 megawatts”

(3) citations, e.g., “10 U.S. 100” or “1998 S. Ct. 1”

(4) companies, e.g., “Lexpredict LLC”

(5) conditions, e.g., “subject to …” or “unless and until …”

(6) constraints, e.g., “no more than”

(7) copyright, e.g., “(C) Copyright 2000 Acme”

(8) courts, e.g., “Supreme Court of New York”

(9) CUSIP, e.g., “392690QT3”

(10) dates, e.g., “June 1, 2017” or “2018-01-01”

(11) definitions, e.g., “Term shall mean …”

(12) distances, e.g., “fifteen miles”

(13) durations, e.g., “ten years” or “thirty days”

(14) geographic and geopolitical entities, e.g., “New York” or “Norway”

(15) money and currency usages, e.g., “$5” or “10 Euro”

(16) percents and rates, e.g., “10%” or “50 bps”

(17) PII, e.g., “212-212-2121” or “999-999-9999”

(18) ratios, e.g.,” 3:1” or “four to three”

(19) regulations, e.g., “32 CFR 170”

(20) trademarks, e.g., “MyApp (TM)”

(21) URLs, e.g., “http://acme.com/”

(22) addresses, e.g., “1999 Mount Read Blvd, Rochester, NY, USA, 14615”

(23) persons, e.g., “John Doe

In [30]:
# !git clone https://github.com/LexPredict/lexpredict-lexnlp.git
# !bash -c 'mv lexpredict-lexnlp/lexnlp .'
# !pip install -r lexpredict-lexnlp/python-requirements.txt
import requests,os
import pandas,urllib.request
import nltk
# import lexnlp
import lexnlp.extract.en.entities.nltk_re
from lexnlp.extract.en.citations import get_citations, get_citation_annotations
# import lexnlp.extract.en.citations.get_citations
import lexnlp.extract.en.acts
import lexnlp.extract.en.conditions
import lexnlp.extract.en.constraints
import lexnlp.extract.en.copyright
import lexnlp.extract.en.courts
from lexnlp.extract.en.dict_entities import entity_config, add_alias_to_entity
from lexnlp.extract.en.cusip import get_cusip_list
import lexnlp.extract.en.dates
import lexnlp.extract.en.definitions
import lexnlp.extract.en.distances
from lexnlp.extract.en.geoentities import get_geoentities, load_entities_dict_by_path
import lexnlp.extract.en.money
import lexnlp.extract.en.percents
import lexnlp.extract.en.pii
import lexnlp.extract.en.ratios
import lexnlp.extract.en.regulations
import lexnlp.extract.en.trademarks
import lexnlp.extract.en.urls

nltk.download('punkt')
nltk.download('averaged_perceptron_tagger')
nltk.download('wordnet')

url = "https://raw.githubusercontent.com/unt-iialab/INFO5731_Spring2020/master/In_class_exercise/01-05-1%20%20Adams%20v%20Tanner.txt"
resp = requests.get(url)
text = resp.text



#acts
print(lexnlp.extract.en.acts.get_act_list(text))

#amounts
print(list(lexnlp.extract.en.amounts.get_amounts(text)))

# citations
print(list(lexnlp.extract.en.citations.get_citations(text)))

#companies
print(list(lexnlp.extract.en.entities.nltk_re.get_companies(text)))

# conditions
print(list(lexnlp.extract.en.conditions.get_conditions(text)))

# constraints
print(list(lexnlp.extract.en.constraints.get_constraints(text)))

# copyright
print(list(lexnlp.extract.en.copyright.get_copyright(text)))

# COURTS
court_df = pandas.read_csv("https://raw.githubusercontent.com/LexPredict/lexpredict-legal-dictionary/1.0.5/en/legal/us_courts.csv")
# Create config objects
court_config_data = []
for _, row in court_df.iterrows():
    c = entity_config(row["Court ID"], row["Court Name"], 0, row["Alias"].split(";") if not pandas.isnull(row["Alias"]) else [])
    court_config_data.append(c)
for entity, alias in lexnlp.extract.en.courts.get_courts(text, court_config_data):
    print("entity=", entity)
    print("alias=", alias)
entity= (98, 'Eastern District of Virginia', 0, [('Eastern District of Virginia', None, False, None), ('E.D. Va.', None, False, None)])
alias= ('E.D. Va.', None, False, None)
entity= (70, 'Southern District of New York', 0, [('Southern District of New York', None, False, None), ('S.D.N.Y.', None, False, None)])
alias= ('S.D.N.Y.', None, False, None)

# CUSIP
print(lexnlp.extract.en.cusip.get_cusip_list(text))

# DATES
print(list(lexnlp.extract.en.dates.get_dates(text)))

# DEFINITIONS
print(list(lexnlp.extract.en.definitions.get_definitions(text)))

# DISTANCES
base_path = os.getcwd()
entities_fn = os.path.join(base_path, 'geoentities.csv')
aliases_fn = os.path.join(base_path, 'geoaliases.csv')
loaded = load_entities_dict_by_path(entities_fn, aliases_fn)
print(list(get_geoentities(text, geo_config_list=loaded)))

# MONEY
print(list(lexnlp.extract.en.money.get_money(text)))

# PERCENTS
print(list(lexnlp.extract.en.percents.get_percents(text)))

# PII
print(list(lexnlp.extract.en.pii.get_pii(text)))

# RATIO
print(list(lexnlp.extract.en.ratios.get_ratios(text)))

# REGULATIONS
print(list(lexnlp.extract.en.regulations.get_regulations(text)))

# TRADEMARKS
print(list(lexnlp.extract.en.trademarks.get_trademarks(text)))

# URLS
print(list(lexnlp.extract.en.urls.get_urls(text)))




[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[]
[5.0, 740.0, 1843.0, 2.0, 1.0, 4.0, 2.0, 1821.0, 5.0, 1.0, 1840.0, 3777, 80.0, 100.0, 30, 1839.0, 741.0, 22, 1840.0, 14000, 120, 1, 1840.0, 3, 4, 1, 1.0, 1840.0, 2.0, 1.0, 361.0, 1.0, 307.0, 6.0, 604.0, 1.0, 2.0, 418.0, 422.0, 7.0, 34.0, 41.0, 167.0, 742.0, 3.0, 112.0, 207.0, 3.0, 338.0, 424.0, 5.0, 26.0, 13.0, 235.0, 8.0, 693.0, 4.0, 1821.0, 167.0, 2.0, 2.0, 216.0, 3.0, 66.0, 4.0, 130.0, 29.0, 2.0, 241.0, 2.0, 332.0, 2.0, 422.0, 9.0, 112.0, 743.0, 9.0, 39.0, 14000, 1840.0, 744.0, 5.0, 182.0, 3.0, 368.0, 1.0, 397.0, 6.0, 604.0, 1, 1821.0, 167.0, 745.0, 4.0, 746.0, 4.0, 210.0, 